In [4]:
import sqlite3
import json
from pathlib import Path

# Setup
db_path = Path("quran_translations.db")
json_output_path = Path("toc/surah_list.json")

# Connect to DB
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Try to detect table with relevant columns
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
tables = [t[0] for t in tables]

# Try to find a matching table
target_table = None
for table in tables:
    cols = [col[1] for col in cursor.execute(f"PRAGMA table_info({table})")]
    if {'surah_no', 'surah_name_ar', 'surah_name_roman', 'surah_name_en'}.issubset(set(cols)):
        target_table = table
        break

if not target_table:
    conn.close()
    raise ValueError("Could not find a table with required columns.")

# Fetch data
cursor.execute(f"SELECT surah_no, surah_name_ar, surah_name_roman, surah_name_en FROM {target_table}")
rows = cursor.fetchall()

# Create list of dicts
surah_data = [
    {
        "surah_no": row[0],
        "surah_name_ar": row[1],
        "surah_name_roman": row[2],
        "surah_name_en": row[3]
    }
    for row in rows
]

# Ensure output directory exists
json_output_path.parent.mkdir(parents=True, exist_ok=True)

# Write JSON
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(surah_data, f, ensure_ascii=False, indent=2)

# Cleanup
conn.close()

# Return relative path
json_output_path.name


'surah_list.json'